# Задание 1. SQL
Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему `covid` меньше всего раз относительно других городов.
####  Как подключиться к PostgreSQL через psycopg2:
1. Импортируем библиотеку.
2. Далее используем данные для входа

In [1]:
# import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# A long string that contains the necessary Postgres login information 
postgres_str = ('postgresql://{username}:{password}@{host}:{port}/{dbname}'\
                .format(username='robot-startml-ro', \
                   password='pheiph0hahj1Vaif', \
                   host='postgres.lab.karpov.courses', \
                   port=6432, \
                   dbname='startml')) 

# Create the connection 
cnx = create_engine(postgres_str)
pd.read_sql_query('SELECT * FROM "user" limit 1;', cnx).head(1)

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads


В конце работы необходимо закрывать курсор и соединение с БД:

In [ ]:
cursor.close() 
connection.close()

#### Как подключиться к PostgreSQL через pandas:

In [5]:
import pandas as pd

In [4]:
# import pandas as pd

df = pd.read_sql(
    """SELECT * FROM "feed_action" LIMIT 10 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.head()

,user_id,post_id,action,time
0,139546,2332,view,2021-11-21 23:58:16
1,109081,2483,view,2021-11-21 23:58:16
2,32770,2529,view,2021-11-21 23:58:16
3,32571,2537,view,2021-11-21 23:58:16
4,134004,2555,view,2021-11-21 23:58:16


### В базе данных имеется три таблицы:

#### 1. Таблица с данными о пользователях (`user`):

`id` - уникальный идентификатор пользователя (primary key)

`gender` - пол

`age` - возраст

`country` - страна

`city` - город

`exp_group` - экспериментальная группа

`os` - операционная система

`source` - источник трафика

#### 2. Таблица с данными о постах (`post`):

`id` - уникальный идентификатор поста (primary key)

`text` - текст поста

`topic` - тема поста

#### 3. Таблица с данными о действиях пользователей (`feed_action`):

`user_id` (——>) `user (id)` - идентификатор пользователя     

`post_id` (——>) `post (id)` - идентификатор поста     

`action` - совершенное в сети действие     

`time` - время действия

# Задание 1. SQL
Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему `covid` меньше всего раз относительно других городов.

1. Получить пользователей и количество просмотренных записей на тему ковида
2. Сгруппировать по городам
3. взять минимум среди всех городов
4. отфильтровать по возрасту и дате
5. если после фильтрации ничего нет, передвинуть фильтрацию на шаг раньше

In [37]:
city_min_view_covid = pd.read_sql(f"""
        SELECT
            city_count.city
        FROM (
                SELECT 
                    city,
                    count(*) as view_number
                FROM "user" u
                INNER JOIN "feed_action" fa on
                    u.id = fa.user_id
                INNER JOIN "post" p on
                    fa.post_id = p.id 
                WHERE 1=1
                AND action = 'view'
                AND topic = 'covid'
                AND age = 36
                AND date(time) = '2021-12-01'
                GROUP BY city
                ORDER BY count(*)
                LIMIT 1
        ) as city_count
        """, con = cnx)

In [38]:
city_min_view_covid

,city
0,Arkhangelsk


# Задание 2. Статистика
Воспользуемся доверительными интервалами, чтобы сделать более точные выводы о метриках модели.

Для начала скачаем данные:

In [1]:
from sklearn.datasets import fetch_california_housing
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

Это задача предсказания средней цены в области на дом. Давайте оценим 95-процентный доверительный интервал на MSE Ridge-регрессии для этой выборки. Будем использовать бутстрэп для этого.

Для этого в цикле будем генерировать бутстрэп-выборки для обучения модели (то есть брать объекты из выборки с возвращением). В качестве тестовых объектов будем брать оставшиеся объекты. Каждый раз будем обучать модель и оценивать качество.

Реализуйте такой код для 95-процентного доверительного интервала на MSE.

Используйте 1000 итераций генераций выборки и обучения модели. Данные не обрабатывайте. Полезными методами будут `np.random.choice` для генерации индексов бутстрэп-выборок (не забудьте про «генерацию с возвращением»), `np.setdiff1d` для получения остатка выборки.

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

plt.style.use('ggplot')

/Users/antonmazilin/study_projects/karpov-courses-ml/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
def get_bootstrap(
    X,
    y,
    boot_it = 1000, # количество бутстрэп-подвыборок
    bootstrap_conf_level = 0.95, # уровень значимости
    alpha = 1 # параметр Ridge reg
):
    # Collect mse error
    boot_data = []
    ridgereg = Ridge(alpha=alpha)
    # Set range for random choice
    boot_len = len(X)
    # Set range for setdiff1d choice
    oob_scope = np.arange(boot_len)
    for i in tqdm(range(boot_it)): # извлекаем подвыборки

        # Make dataset train—test split
        bootstrap_indeces = np.random.choice(boot_len, boot_len, replace=True)
        X_train = X.iloc[bootstrap_indeces]
        y_train = y[bootstrap_indeces]
        
        # Create indices for the out-of-bag data using np.setdiff1d
        oob_indices = np.setdiff1d(oob_scope, bootstrap_indeces, assume_unique=False)
        X_test = X.iloc[oob_indices]
        y_test = y[oob_indices]

        ridgereg.fit(X_train, y_train)
        y_pred = ridgereg.predict(X_test)
    
        mse = mean_squared_error(y_test, y_pred) # считаю ошибку
        
        boot_data.append(mse) # складываю ошибки в список
        
    pd_boot_data = pd.DataFrame(boot_data, columns=['error'])
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    ci = pd_boot_data.quantile([left_quant, right_quant])
    # sns.histplot(pd_boot_data, kde=True, stat="density")

    return {"boot_data": boot_data, 
            "ci": ci}

In [50]:
bootstrap_dict = get_bootstrap(X, y)

100%|██████████| 1000/1000 [00:08<00:00, 115.28it/s]


In [54]:
bootstrap_dict['ci'].reset_index().rename(columns={'index': 'quantile'})

,quantile,error
0,0.025,0.506546
1,0.975,1.251516


# Задание 3. Machine learning
Чему равно значение метрики AUC ROC у классификатора, который для любого объекта возвращает значение 0.97, если доля положительного класса в выборке составляет 4%?     
Опишите свои рассуждения как можно точнее, с расчетами, если они потребуются.

**Ответ**
- Классификатор возвращает высокую вероятность того, что все объекты принадлежат положительному классу 1, хотя истинно только 4% принадлежат положительному классу 1.
- Что означает, что классификатор неправильно предсказывает метки отрицательного класса 0
- FPR будет при этом высоким значением, так как классификатор отрицательные метки класса, предсказывает положительными
- TPR будет также высоким, тк классификатор правильно определяет положительный класс

**Вывод**: классификатор будет являться слабым, точка на графике 0.97 при истинно положительных 4% в осях TPR/FPR будет находится в правом верхнем углу, при этом площадь (метрика AUC ROC) будет около 0.5

**Рассуждения**
- Значение метрики AUC (area under curve) — является площадь под ROC кривой, где ROC кривая в осях TPR/FPR;
- На координатной плоскости кривая стартует с координатами (0, 0) и приходит в координаты (1, 1);
- Метрика отражает способность классификатора различать классы со значение от 0,5 (случайная классификация), до 1.0 (идеальная классификация);
- Чем лучше классификатор разделяет два класса, тем больше площадь (area under curve) под ROC-кривой;

Доля положительных объектов правильно предсказанных положительными

- TPR(true positive rate) = TP / TP + FN

Доля неправильно предсказанных положительных объектов

- FPR(false positive rate) = FP / TN + FP

# Задание 4. Deep learning
Напишите функцию `get_pretrained_model`, которая принимает в качестве аргументов название архитектуры, количество классов для задачи классификации и стоит ли инициализировать модель с помощью полученных в ходе обучения на датасете `ImageNet`. Она должна иметь следующую сигнатуру: `def get_pretrained_model(model_name: str, num_classes: int, pretrained: bool=True):`

Будем считать, что на вход могут прийти четыре различных model_name: `alexnet`, `vgg11`, `googlenet` и `resnet18`. Для каждого из них нужно вернуть соответствующую модель из зоопарка моделей `torchvision`.

Чтобы понять, как именно модифицировать созданные объекты, посмотрите на исходный код для моделей:

https://pytorch.org/hub/pytorch_vision_resnet/    
https://pytorch.org/hub/pytorch_vision_alexnet/    
https://pytorch.org/hub/pytorch_vision_vgg/    
https://pytorch.org/hub/pytorch_vision_googlenet/

\-